In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set up directories and image size
train_dir = 'organized_data/train'
test_dir = 'organized_data/validation'
img_size = 224  # Standard VGG16 input size
batch_size = 64

# Define the image data generators for loading the dataset
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)  # Added horizontal flip for augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load the VGG16 model with pre-trained weights, excluding the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze all layers of the base model first
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last 4 layers for fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Add custom classification layers on top of VGG16
model = models.Sequential()
model.add(base_model)  # Add the pre-trained VGG16 base
model.add(layers.GlobalAveragePooling2D())  # Global average pooling instead of flattening
model.add(layers.Dense(512, activation='relu'))  # Fully connected layer with 512 units
model.add(layers.Dropout(0.5))  # Adding dropout for regularization
model.add(layers.Dense(9, activation='softmax'))  # New output layer with 9 classes

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks for learning rate scheduling and early stopping
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Train the model with callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Initial epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[early_stopping, lr_scheduler]
)

# Fine-tune the entire model by unfreezing all layers for additional training
for layer in base_model.layers:
    layer.trainable = True

# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune with additional epochs
fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Fine-tuning epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluate the model on the test set for top-1 accuracy
test_loss, test_acc = model.evaluate(test_generator)
print(f'Transfer Learning with VGG16 Test Accuracy after Fine-Tuning: {test_acc:.4f}')

Found 902 images belonging to 9 classes.
Found 1698 images belonging to 9 classes.
Epoch 1/20
14/14 [==============================] - 6s 416ms/step - loss: 1.9518 - accuracy: 0.2745 - val_loss: 1.2819 - val_accuracy: 0.6791
Epoch 2/20
14/14 [==============================] - 5s 379ms/step - loss: 1.0978 - accuracy: 0.6289 - val_loss: 0.5265 - val_accuracy: 0.8365
Epoch 3/20
14/14 [==============================] - 5s 396ms/step - loss: 0.6861 - accuracy: 0.7733 - val_loss: 0.3992 - val_accuracy: 0.8966
Epoch 4/20
14/14 [==============================] - 6s 414ms/step - loss: 0.4144 - accuracy: 0.8496 - val_loss: 0.3222 - val_accuracy: 0.8798
Epoch 5/20
14/14 [==============================] - 5s 388ms/step - loss: 0.3163 - accuracy: 0.8842 - val_loss: 0.2374 - val_accuracy: 0.9279
Epoch 6/20
14/14 [==============================] - 5s 387ms/step - loss: 0.2087 - accuracy: 0.9320 - val_loss: 0.1968 - val_accuracy: 0.9411
Epoch 7/20
14/14 [==============================] - 5s 388ms/step